## For testing a saved raytracing model

In [ ]:
import os
from gpuutils import GpuUtils
GpuUtils.allocate(gpu_count=1, framework='keras')

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

import pandas as pd
from tensorflow import keras
import numpy as np
np.set_printoptions(precision=3, suppress=True)

from matplotlib.ticker import AutoMinorLocator

from matplotlib import pyplot as plt, cm
import seaborn as sns
from radiotools import plthelpers as php

pal = sns.color_palette("colorblind")
print(pal.as_hex())
plt.style.use('plot_style.txt')

### Load the data

In [ ]:
#df_test = pd.read_csv('/mnt/md0/aholmberg/data/raytrace_samples_random.csv')
df_test = pd.read_pickle('/mnt/md0/aholmberg/data/raytracing_random_spherical.pkl')

### Load the model

In [ ]:
model_name = 'ResNetFc_w50_d14_vwidth20_vdepth2_news_lrelu_ln_179deg_250_np'
model_path = '/mnt/md0/aholmberg/models/' + model_name
path_to_loss = 'losses/history_' + model_name + '.pkl'
model = keras.models.load_model(model_path)
print(model.summary())

### Load and find minimum loss 

In [ ]:
history = pd.read_pickle(path_to_loss)

In [ ]:
idx = history['val_loss'].idxmin()
print(history['val_dense_1_loss'].iloc[idx])
print(history['val_loss'].iloc[idx])

### Plot the losses of the different outputs

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_yscale('log')
ax.set_xlabel('epoch')
ax.set_ylabel('MSE loss')
for i in range(1,5):
    ax.plot(history[f'dense_{i}_loss'])
    ax.plot(history[f'val_dense_{i}_loss'])

### Define and scale the test data

In [ ]:
""" sc_pos_r = df_test['source_pos_r'].to_numpy().astype(np.float32)
sc_pos_z = df_test['source_pos_z'].to_numpy().astype(np.float32)
ant_pos_z = df_test['antenna_pos_z'].to_numpy().astype(np.float32)
x_test = np.stack((sc_pos_r, sc_pos_z, ant_pos_z), axis=1)

travel_time = df_test['travel_time'].to_numpy().astype(np.float32)
path_length = df_test['path_length'].to_numpy().astype(np.float32)
launch = df_test['launch_angle'].to_numpy().astype(np.float32)
recieve = df_test['recieve_angle'].to_numpy().astype(np.float32)
y_test = np.stack((travel_time, path_length, launch, recieve), axis=1)

type = df_test['type'].to_numpy().astype(np.float32)


unique, index, count = np.unique(x_test, return_counts=True, return_index=True, axis=0)

type = np.delete(type, index[count == 1], axis=0)
type_1 = type[0::2]
type_2 = type[1::2]

#x_test[index[count == 1], :]
x_new_test = np.delete(x_test, index[count == 1], axis=0)
y_new_test = np.delete(y_test, index[count == 1], axis=0)
unique, index, count = np.unique(x_new_test, return_counts=True, return_index=True, axis=0) """

sc_pos_d = df_test['source_pos_d'].to_numpy().astype(np.float64)
sc_pos_phi = df_test['source_pos_phi'].to_numpy().astype(np.float64)
ant_pos_z = df_test['antenna_pos_z'].to_numpy().astype(np.float64)
x = np.stack((sc_pos_d, sc_pos_phi, ant_pos_z), axis=1)


travel_time = df_test['travel_time'].to_numpy().astype(np.float64)
path_length = df_test['path_length'].to_numpy().astype(np.float64)
launch = df_test['l_angle'].to_numpy().astype(np.float64)
recieve = df_test['r_angle'].to_numpy().astype(np.float64)
y = np.stack((travel_time, path_length, launch, recieve), axis=1)

unique, index, count = np.unique(x, return_counts=True, return_index=True, axis=0)
x_new_test = np.delete(x, index[count == 1], axis=0)
y_new_test = np.delete(y, index[count == 1], axis=0)

type = df_test['type'].to_numpy().astype(np.float32)
type = np.delete(type, index[count == 1], axis=0)
type_1 = type[0::2]
type_2 = type[1::2]

x_test = x_new_test[0::2,:]

y_temp1_test  = y_new_test[0::2,:]
y_temp2_test  = y_new_test[1::2,:]
y_new_test = np.zeros((y_temp1_test.shape[0], 8))


for i in range(4):
    y_new_test[:,2*i] = y_temp1_test[:,i]
    y_new_test[:,2*i+1] = y_temp2_test[:,i]


y_test = y_new_test

index = x_test[:, 1] > 179
x_test = np.delete(x_test, index, axis=0)
y_test = np.delete(y_test, index, axis=0)
type_1 = np.delete(type_1, index, axis=0)
type_2 = np.delete(type_2, index, axis=0)

norm_x_test = np.zeros_like(x_test)
norm_x_test[:, 0] = x_test[:, 0] / (np.sqrt(2700**2 + 2000**2))
norm_x_test[:, 1] = x_test[:, 1] / (180)
norm_x_test[:, 2] = x_test[:, 2] / -(200)

norm_y_test = np.zeros_like(y_test)
norm_y_test[:, 0] = y_test[:, 0] / (20000)
norm_y_test[:, 1] = y_test[:, 1] / (21000)
norm_y_test[:, 2] = y_test[:, 2] / (3500)
norm_y_test[:, 3] = y_test[:, 3] / (3800)
norm_y_test[:, 4] = y_test[:, 4] / (180)
norm_y_test[:, 5] = (y_test[:, 5] - 90) / (90)
norm_y_test[:, 6] = y_test[:, 6] / (180)
norm_y_test[:, 7] = y_test[:, 7] / (90)

In [ ]:
type_sol_1 = np.array(['direct' if x == 1 else 'refracted' for x in type_1])
type_sol_2 = np.array(['reflected' if x == 3 else 'refracted' for x in type_2])

### Predict quantities of interest and inverse scale them

In [ ]:
temp1 = model(norm_x_test[:int(norm_x_test.shape[0]/2)])
temp2 = model(norm_x_test[int(norm_x_test.shape[0]/2):])
print(temp1[0].shape ,temp1[1].shape, temp1[2].shape, temp1[3].shape,temp2[0].shape ,temp2[1].shape, temp2[2].shape)
y_test_pred = np.zeros((norm_y_test.shape))

y_test_pred[:int(norm_x_test.shape[0]/2),:] = np.concatenate((temp1[0], temp1[1], temp1[2], temp1[3]), axis=1)
y_test_pred[int(norm_x_test.shape[0]/2):,:] = np.concatenate((temp2[0], temp2[1], temp2[2], temp2[3]), axis=1)

y_test_inv = np.zeros_like(y_test_pred)
y_test_inv[:, 0] = y_test_pred[:, 0] * (20000)
y_test_inv[:, 1] = y_test_pred[:, 1] * (21000)
y_test_inv[:, 2] = y_test_pred[:, 2] * (3500)
y_test_inv[:, 3] = y_test_pred[:, 3] * (3800)
y_test_inv[:, 4] = y_test_pred[:, 4] * (180)
y_test_inv[:, 5] = y_test_pred[:, 5] * 90 + 90
y_test_inv[:, 6] = y_test_pred[:, 6] * (180)
y_test_inv[:, 7] = y_test_pred[:, 7] * (90)

print(y_test_pred.shape)
y_time = y_test_pred[:,:2]
y_length = y_test_pred[:,2:4]
y_launch = y_test_pred[:,4:6]
y_recieve = y_test_pred[:,6:8]

### Find the error, compute statsand make a table

In [ ]:
diff_deg = y_test - y_test_inv

sol = ['time_sol_1:', 
       'time_sol_2:',
       'length_sol_1:',
       'length_sol_2:',
       'launch_sol_1:',
       'launch_sol_2:',
       'recieve_sol_1:',
       'recieve_sol_2:']

tab = np.zeros((8,5))
for i in range(8):
    mean = np.mean(diff_deg[:,i])
    med = np.median(diff_deg[:,i])
    std = np.std(diff_deg[:,i])
    per = np.percentile(diff_deg[:,i], (16, 84))
    print(sol[i] + f' mean: {mean:.4f} median: {med:.4f}  std: {std:.4f}  percentile 16: {per[0]:.4f} percentile 84: {per[1]:.4f}')
    l = [mean, med, std, per[0], per[1]]
    tab[i,:] = l

temp = pd.DataFrame(tab)

In [ ]:
print(temp.to_latex(header=['mean', 'median', '$\sigma$', '16%', '84%'], float_format='%.3E'))

### Try 2-d histogram of error

In [ ]:
data = pd.DataFrame({'x-val':y_test[:,0] , 'y-val':(y_test_inv[:,0] - y_test[:,0]), 'type':type_sol_1})
sns.histplot(data, x="x-val", y="y-val", palette=['#0173b2', '#de8f05'], hue=type)

### scatter error vs true value with distributions of the point on the margins

In [ ]:
plt.rcParams.update({'font.size': 14})
#sns.set_theme()
for sol in range (8):
    if sol%2 == 0:
        type = type_sol_1
    else:
        type = type_sol_2
    data = pd.DataFrame({'x-val':y_test[:,sol] , 'y-val':(y_test_inv[:,sol] - y_test[:,sol]), 'type':type})

    if sol%2 == 0:
        g = sns.JointGrid(data=data,
                    x="x-val",
                    y="y-val",
                    hue=type,
                    palette=['#0173b2', '#de8f05'],
                    space=0)
    else:
        g = sns.JointGrid(data=data,
                    x="x-val",
                    y="y-val",
                    hue=type,
                    palette=['#029e73', '#de8f05'],
                    space=0)
    
    g.figure.set_size_inches((5,4))
    g.ax_joint.xaxis.set_minor_locator(  AutoMinorLocator(5))
    g.ax_joint.yaxis.set_minor_locator(  AutoMinorLocator(5))
    g.plot_joint(sns.scatterplot, s=10, alpha=.4)
    g.ax_joint.legend(frameon=False)
    g.plot_marginals(sns.histplot, element="step", fill=False)
    #g.ax_marg_y.set_ticks_off()
    if sol == 0 or sol == 1:
        g.set_axis_labels(xlabel='True traveltime [ns]', ylabel='Error travel time [ns]')
    elif sol == 2 or sol == 3:
        g.set_axis_labels(xlabel='True path length [m]', ylabel='Error path length [m]')
    elif sol == 4 or sol == 5:
        g.set_axis_labels(xlabel='True launch angle [$^\circ$]', ylabel='Error launch angle [$^\circ$]')
    elif sol == 6 or sol == 7:
        g.set_axis_labels(xlabel='True receive angle [$^\circ$]', ylabel='Error receive angle [$^\circ$]')
    #g.savefig(f'thesis/Exjobb-rapport/figures/sol_{sol}_simpler.png', dpi=300)

### Try kde plot zoomed in

In [ ]:
plt.rcParams.update({'font.size': 14})

print(y_test_inv.shape, type_sol_1.shape)
for sol in range (1):
    if sol%2 == 0:
        type = type_sol_1
    else:
        type = type_sol_2
    data = pd.DataFrame({'x-val':y_test[:,sol] , 'y-val':(y_test_inv[:,sol] - y_test[:,sol]), 'type':type})
    lim = 3*np.std(y_test_inv[:,sol] - y_test[:,sol])
    if sol%2 == 0:
        g = sns.JointGrid(data=data,
                    x="x-val",
                    y="y-val",
                    hue=type,
                    ylim=(-lim,lim),
                    palette=['#0173b2', '#de8f05'],
                    space=0)
    else:
        g = sns.JointGrid(data=data,
                    x="x-val",
                    y="y-val",
                    hue=type,
                    ylim=(-lim,lim),
                    palette=['#029e73', '#de8f05'],
                    space=0)
    
    
    g.figure.set_size_inches((5,4))
    g.ax_joint.xaxis.set_minor_locator(  AutoMinorLocator(5))
    g.ax_joint.yaxis.set_minor_locator(  AutoMinorLocator(5))
    g.plot_joint(sns.kdeplot, s=10, alpha=.4)
    g.plot_marginals(sns.histplot)
    if sol == 0 or sol == 1:
        g.set_axis_labels(xlabel='True traveltime [ns]', ylabel='Error travel time [ns]')
    elif sol == 2 or sol == 3:
        g.set_axis_labels(xlabel='True path length [m]', ylabel='Error path length [m]')
    elif sol == 4 or sol == 5:
        g.set_axis_labels(xlabel='True launch angle [$^\circ$]', ylabel='Error launch angle [$^\circ$]')
    elif sol == 6 or sol == 7:
        g.set_axis_labels(xlabel='True recieve angle [$^\circ$]', ylabel='Error recieve angle [$^\circ$]')
    #g.savefig(f'/mnt/md0/aholmberg/plots/raytrace/jointgrid/sol_zoomed_{sol}_simpler.png')

### plot histograms of the error

In [ ]:
x_lab = ['travel time [ns]', 'path length [m]', 'launch angle $[^\circ]$', 'recieve angle $[^\circ]$']
for i in range(8):
    std = np.std(diff_deg[:,i])
    bins = np.linspace(-3*std, 3*std, 100)
    fix, ax = php.get_histogram(diff_deg[:,i], bins=bins)
    x_label = 'Error in ' + x_lab[int(i/2)]
    plt.xlabel(x_label)
    #plt.savefig(f'/mnt/md0/aholmberg/plots/raytrace/jointgrid/hist_sol_{i}.png')

### Plot outliers to find potential pattern

In [ ]:
sol = 0
if sol%2 == 0:
    type = type_sol_1
else:
    type = type_sol_2

err = y_test_inv[:,sol] - y_test[:,sol]
index_of_outliers = np.abs(err) > 2  # arbitrary
out_err = err[index_of_outliers]
out_true = y_test[index_of_outliers,sol]
out_type = type[index_of_outliers]
out_d = x_test[index_of_outliers, 0]
out_phi = x_test[index_of_outliers, 1]
out_z = x_test[index_of_outliers, 2]
data = pd.DataFrame({'distance':out_d, 'angle':out_phi, 'z-pos':out_z, 'true_type':out_type})
#'true_time':out_true , 'error':out_err, 
print(data.shape)
data.head()

In [ ]:
g = sns.pairplot(data=data, hue='true_type')
#g.savefig('plots/length_sol2_1m_outliers_179_5deg.png')

# Make heatmap
### Ploting heatmap of error using a test set with fixed antenna position
The code for binning the error is very slow ~1h so should be optimised

In [ ]:
df_test = pd.read_pickle('/mnt/md0/aholmberg/data/raytrace_random_17_antenna_1_spherical.pkl')

In [ ]:
sc_pos_d = df_test['source_pos_d'].to_numpy().astype(np.float64)
sc_pos_phi = df_test['source_pos_phi'].to_numpy().astype(np.float64)
ant_pos_z = df_test['antenna_pos_z'].to_numpy().astype(np.float64)
x = np.stack((sc_pos_d, sc_pos_phi, ant_pos_z), axis=1)


travel_time = df_test['travel_time'].to_numpy().astype(np.float64)
path_length = df_test['path_length'].to_numpy().astype(np.float64)
launch = df_test['l_angle'].to_numpy().astype(np.float64)
recieve = df_test['r_angle'].to_numpy().astype(np.float64)
y = np.stack((travel_time, path_length, launch, recieve), axis=1)

unique, index, count = np.unique(x, return_counts=True, return_index=True, axis=0)
x_new_test = np.delete(x, index[count == 1], axis=0)
y_new_test = np.delete(y, index[count == 1], axis=0)

type = df_test['type'].to_numpy().astype(np.float32)
type = np.delete(type, index[count == 1], axis=0)
type_1 = type[0::2]
type_2 = type[1::2]

x_test = x_new_test[0::2,:]

y_temp1_test  = y_new_test[0::2,:]
y_temp2_test  = y_new_test[1::2,:]
y_new_test = np.zeros((y_temp1_test.shape[0], 8))


for i in range(4):
    y_new_test[:,2*i] = y_temp1_test[:,i]
    y_new_test[:,2*i+1] = y_temp2_test[:,i]


y_test = y_new_test

index = x_test[:, 1] > 179
x_test = np.delete(x_test, index, axis=0)
y_test = np.delete(y_test, index, axis=0)
type_1 = np.delete(type_1, index, axis=0)
type_2 = np.delete(type_2, index, axis=0)

norm_x_test = np.zeros_like(x_test)
norm_x_test[:, 0] = x_test[:, 0] / (np.sqrt(2700**2 + 2000**2))
norm_x_test[:, 1] = x_test[:, 1] / (180)
norm_x_test[:, 2] = x_test[:, 2] / -(200)

norm_y_test = np.zeros_like(y_test)
norm_y_test[:, 0] = y_test[:, 0] / (20000)
norm_y_test[:, 1] = y_test[:, 1] / (21000)
norm_y_test[:, 2] = y_test[:, 2] / (3500)
norm_y_test[:, 3] = y_test[:, 3] / (3800)
norm_y_test[:, 4] = y_test[:, 4] / (180)
norm_y_test[:, 5] = (y_test[:, 5] - 90) / (90)
norm_y_test[:, 6] = y_test[:, 6] / (180)
norm_y_test[:, 7] = y_test[:, 7] / (90)

In [ ]:
type_sol_1 = np.array(['direct' if x == 1 else 'refracted' for x in type_1])
type_sol_2 = np.array(['reflected' if x == 3 else 'refracted' for x in type_2])

In [ ]:
temp1 = model(norm_x_test[:int(norm_x_test.shape[0]/2)])
temp2 = model(norm_x_test[int(norm_x_test.shape[0]/2):])
y_test_pred = np.zeros((norm_y_test.shape))

y_test_pred[:int(norm_x_test.shape[0]/2),:] = np.concatenate((temp1[0], temp1[1], temp1[2], temp1[3]), axis=1)
y_test_pred[int(norm_x_test.shape[0]/2):,:] = np.concatenate((temp2[0], temp2[1], temp2[2], temp2[3]), axis=1)

# y_test_inv = scaler_y.inverse_transform(y_test_pred)


y_test_inv = np.zeros_like(y_test_pred)
y_test_inv[:, 0] = y_test_pred[:, 0] * (20000)
y_test_inv[:, 1] = y_test_pred[:, 1] * (21000)
y_test_inv[:, 2] = y_test_pred[:, 2] * (3500)
y_test_inv[:, 3] = y_test_pred[:, 3] * (3800)
y_test_inv[:, 4] = y_test_pred[:, 4] * (180)
y_test_inv[:, 5] = y_test_pred[:, 5] * 90 + 90
y_test_inv[:, 6] = y_test_pred[:, 6] * (180)
y_test_inv[:, 7] = y_test_pred[:, 7] * (90)

y_time = y_test_pred[:,:2]
y_length = y_test_pred[:,2:4]
y_launch = y_test_pred[:,4:6]
y_recieve = y_test_pred[:,6:8]



In [ ]:
diff = y_test - y_test_inv


sol = ['time_sol_1:', 
       'time_sol_2:',
       'length_sol_1:',
       'length_sol_2:',
       'launch_sol_1:',
       'launch_sol_2:',
       'recieve_sol_1:',
       'recieve_sol_2:']

for i in range(8):
    mean = np.mean(diff[:,i])
    std = np.std(diff[:,i])
    print(sol[i] + f' mean: {mean:.4f}  std: {std:.4f}')

In [ ]:
print(x_test[0, :])
x = x_test[0, 0]*np.cos(np.deg2rad(x_test[0, 1] + 90))
y = x_test[0, 0]*np.sin(np.deg2rad(x_test[0, 1] + 90)) - 1 # add antenna pos
d = np.sqrt(x**2 + (y + 1)**2)
print(d)
print(x, y)

l_bounds = [-2000, -2700, -200]
u_bounds = [-1, -1, -1]
vals = np.zeros((2650, 1950, 2, 8))
coords = np.zeros((2650, 1950, 2))

antenna_pos = -1

x_test_cart = np.zeros_like(x_test) # change to cartesian coords
x_test_cart[:, 0] = x_test[:, 0]*np.cos(np.deg2rad(x_test[:, 1] + 90))
x_test_cart[:, 1] = x_test[:, 0]*np.sin(np.deg2rad(x_test[:, 1] + 90)) + antenna_pos
x_test_cart[:, 2] = x_test[:, 2]

In [ ]:

for i in range(-1,-2001+50, -1):
    for j in range(-1, -2701+50, -1):
        coords[2650 + j, -(i+1), :] = [i - 25, j - 25] # mid    point
        temp_r = np.logical_and(x_test_cart[:,0] < i, x_test_cart[:,0] > i - 50)
        temp_z = np.logical_and(x_test_cart[:,1] < j, x_test_cart[:,1] > j - 50)
        binned_vals = diff[np.logical_and(temp_r, temp_z), :]
        if binned_vals.shape[0] >= 10:
            vals[2650 + j, -(i+1), 0, :] = np.std(binned_vals, axis=0)
            vals[2650 + j, -(i+1), 1, :] = np.mean(binned_vals, axis=0)
        else:
            vals[2650 + j, -(i+1),: , :] = -100

In [ ]:
import matplotlib.colors as colors

In [ ]:
tmp = np.fliplr(vals[:,:,0,1])

cmap = cm.get_cmap('viridis')
cmap.set_under('white')
extent = [-1975, -26, -2675 , -26]


fig, ax = plt.subplots(1,2, figsize=(8, 5))
ax_im1 = ax[0].imshow(tmp,
                  extent=extent,
                  norm=colors.LogNorm(vmin=0.0001, vmax=tmp.max()),
                  cmap=cmap,
                  #vmin=0,
                  origin='lower',
                  aspect='equal')
fig.colorbar(ax_im1, ax=ax[0], label='log$(\sigma)$ [ns]')
ax[0].set_xlabel('x [m]')
ax[0].set_ylabel('z [m]')
ax_im2 = ax[1].imshow(tmp,
                  extent=extent,
                  cmap=cmap,
                  vmin=0,
                  origin='lower',
                  aspect='equal')
ax[1].set_xlabel('x [m]')
ax[1].set_ylabel('z [m]')
fig.colorbar(ax_im2, ax=ax[1], label='$\sigma$ [ns]')
fig.tight_layout()
#plt.savefig('/mnt/md0/aholmberg/plots/raytrace/jointgrid/heatmap_time_sol2_z50.png')

In [ ]:
cmap = cm.get_cmap('viridis')
cmap.set_under('white')

extent = [-1975, -26, -2675 , -26]
for i in range(8):
    tmp = np.fliplr(vals[:,:,0,0])
    fig, ax = plt.subplots(figsize=(10, 5))

    ax_im = ax.imshow(tmp,
                    extent=extent,
                    cmap=cmap,
                    vmin=0,
                    origin='lower',
                    aspect='equal')
    fig.colorbar(ax_im)
    fig.tight_layout()
    #fig.savefig(f'thesis/Exjobb-rapport/figures/heatmaps/antenna_1_sol{i}.pdf')